**Задача 1.** 
Ознакомиться с описанием датасета, определить целевую переменную (target) и основную задачу прогнозной модели.

**Задача 2.** 
Выполнить этапы разведочного анализа, указанные в блокноте.

**Задача 3.** 
Выполнить творческое задание: попытаться найти признаки (“фичи”), которые будут наиболее полезными для предсказания target-переменной. Использовать любые доступные инструменты разведочного анализа (вычисления, 
визуализация, корреляционный анализ).

In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Подключите необходимые библиотеки.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set();
sns.set(rc={'figure.figsize':(10, 8)}); # you can change this if needed

Загрузите данные. Они находятся в директории input.

In [ ]:
df = pd.read_csv('../input/bigml_59c28831336c6604c800002a.csv', sep=',')

In [ ]:
# Просмотр первых 10-и строк в виде транспонированной таблицы, в связи с большим количеством признаков.
df.head(10).T

In [ ]:
df.info()

In [ ]:
# Статистика по числовым признакам
df.describe().T

Категория - обучение с учителем; тип задачи - классификация (предсказание категории объекта). 

Целевой переменной (target) является churn типа bool. Под churn подразумевается отток клиентов — это потеря клиентов или покупателей. При значении целевой переменной равной "False" имеется в виду, что данный абонент всё ещё является клиентом компании; "True" - абонент отказался от услуг и не является клиентом компании. 

Основная задача: определить лояльных абонентов.


Каково распределение абонентов по целевому признаку **churn**? Назовите процент лояльных абонентов.

In [ ]:
import seaborn as sns
sns.catplot(x = "churn", kind = "count", palette = "ch:.25", data = df, order = df['churn'].value_counts().index)

In [ ]:
# Выведем процентное соотношение
df['churn'].value_counts(normalize=True)

Имеем приблизительно 85.5% лояльных абонентов.

Очевидно, что имеется большая разница в количестве между лояльными и нелояльными абонементами, а это значит, что классы False и True несбалансированы.

In [ ]:
# Сгрупируем данные по штатам, учитывая разные виды звонков (total day minutes, total eve minutes, total night minutes, total intl minutes).
# Добавим новый признак total minutes
df['total minutes'] = df['total day minutes'] + df['total eve minutes'] + df['total night minutes'] + df['total intl minutes']
# df.head().T
df.groupby('state')['total minutes'].sum(axis=1)

Жители какого штата говорят по телефону больше всего? меньше всего? Обратите внимание, что нас интересуют все виды звонков.

In [ ]:
# Отобразим результаты в виде гистограммы
plt.figure(figsize=(15, 8)) # увеличим размер картинки

df.groupby('state')['total minutes'].sum().sort_values(ascending=False).plot(kind='bar', color='green')

plt.xlabel("State", size=12)
plt.ylabel("Total minutes", size=12)
#plt.show();

In [ ]:
df.groupby('state')['total minutes'].sum(axis=1).idxmax()

In [ ]:
df.groupby('state')['total minutes'].sum(axis=1).idxmin()

Максимальное значение - 60789.7, штат WV. Минимальное значение - 20091.0, штат CA.

Верно ли, что люди, совершающие много международных звонков, приобретают **international plan**? Не забывайте о визуализации информации.

In [ ]:
from scipy.stats import chi2_contingency, fisher_exact
chi2_contingency(pd.crosstab(df['total intl calls'], df['international plan']))

Большое значение p-value говорит о том, что связь не является статистически значимой.

Как связаны наличие голосовой почты и международного плана с лояльностью абонентов? Проверьте статистическую значимость этих связей.

In [ ]:
# Построим кросс-таблицу.
pd.crosstab(df['voice mail plan'], df['churn'])

In [ ]:
sns.countplot(x='voice mail plan', hue='churn', data=df, order = df['voice mail plan'].value_counts().index);

In [ ]:
sns.heatmap(pd.crosstab(df['voice mail plan'], df['churn']), cmap="YlGnBu", annot=True, cbar=False);

Вычислим коэффициент $\chi^2$.

In [ ]:
from scipy.stats import chi2_contingency, fisher_exact
chi2_contingency(pd.crosstab(df['voice mail plan'], df['churn']))

In [ ]:
fisher_exact(pd.crosstab(df['voice mail plan'], df['churn']))

Малое значение p-value говорит о том, что связь между наличием голосовой почты и лояльностью абонентов статистически подтверждается.

In [ ]:
# Построим кросс-таблицу.
pd.crosstab(df['international plan'], df['churn'])

In [ ]:
sns.countplot(x='international plan', hue='churn', data=df, order = df['international plan'].value_counts().index);

Легко заметить интересное наблюдение: при подключенном роуминге, доля оттока намного выше. Предпоожительно, большие и плохо контролируемые траты в роуминге очень конфликтогенны и приводят к недовольству клиентов телеком-оператора и, соответственно, к их оттоку.

In [ ]:
sns.heatmap(pd.crosstab(df['international plan'], df['churn']), cmap="YlGnBu", annot=True, cbar=False);

In [ ]:
pd.crosstab(df['churn'], df['international plan'], margins=True)

In [ ]:
chi2_contingency(pd.crosstab(df['international plan'], df['churn']))

In [ ]:
fisher_exact(pd.crosstab(df['international plan'], df['churn']))

Так как p-value принимает малое значение, то связь между наличием международного плана и лояльностью абонентов статистически подтверждается.

Мы отвергаем нулевую гипотезу, т.е. декларируем связь между наличием международного плана и лояльностью абонентов. Также если p-value достаточно мало (до 1% или 5%). При n=30 это соответствует t-статистике равной 2-2.75.

Так же можно заметить, что когда международный план подключен, то оттока намного выше. Вероятно, большие и плохо контролируемые траты в международном плане очень конфликтогенны и приводят к недовольству клиентов и, соответственно, к их оттоку.

Что можно сказать о взаимосвязи количества обращений в колл-центр и лояльности? Не забывайте о визуализации информации.

In [ ]:
# Применяем бисериальный коэффициент корреляции.
from scipy.stats import pointbiserialr

pb2 = pointbiserialr(df['customer service calls'], df['churn'])
print('Point biserialr correlation:', pb2[0], 'p-value:', pb2[1])

In [ ]:
plt.figure(figsize=(20, 20))
sns.countplot(x='customer service calls', hue='churn', data=df, order = df['customer service calls'].value_counts().index)

Так как p-value значительно меньше 0.05, значит, взаимосвязь (корреляция) между количеством обращений в колл-центр и лояльностью абонемента статистически значима. Связь прямая, так как с увеличением или уменьшением значений факторного признака происходит увеличение или уменьшение значений результативного. 

При этом из графика следует, что чем больше обращений в колл-центр, тем выше вероятность того, что абонент откажется от услуг компании. Так как на 4-ом обращении показателли выравниваются, а начиная с 5-го обращения количество нелояльных превышает количество лояльных абонентов.

Так же стоит упомянуть, что количество лояльных абонентов значительно превышает нелояльных в данной выборке, из чего следует что результаты неточные.

Какие из числовых признаков имеют наибольшую степень корреляции? Почему?

In [ ]:
numeric = ['account length', 'number vmail messages', 'total day minutes', 
           'total day calls', 'total day charge', 'total eve minutes','total eve calls', 
           'total eve charge', 'total night minutes', 'total night calls', 'total night charge', 
           'total intl minutes', 'total intl calls', 'total intl charge', 'customer service calls']
sns.pairplot(df[numeric]);

In [ ]:
# Построим матрицу корреляций Спирмена
df[numeric].corr(method='spearman')

In [ ]:
# Визуализируем матрицы
sns.heatmap(df[numeric].corr(method='spearman'));

In [ ]:
df[numeric].corr(method='pearson')

In [ ]:
sns.heatmap(df[numeric].corr(method='pearson'));

Наибольшая корреляция наблюдается между переменными 'total day minutes' и 'total day charge', 'total eve minutes' и 'total eve charge', 'total night minutes' и 'total night charge', 'total intl minutes' и 'total intl charge'. Данные связи являются очевидными, так как стоимость напрямую зависит от длительности звонка. 

Связано ли время пользования аккаунтом с количеством обращений в колл-центр?

In [ ]:
from scipy.stats import pearsonr, spearmanr, kendalltau
r1 = pearsonr(df['account length'], df['customer service calls'])
print('Pearson correlation:', r1[0], 'p-value:', r1[1])

In [ ]:
r2 = spearmanr(df['account length'], df['customer service calls'])
print('Spearman correlation:', r2[0], 'p-value:', r2[1])

In [ ]:
r3 = kendalltau(df['account length'], df['customer service calls'])
print('Kendall correlation:', r3[0], 'p-value:', r3[1])

Так как p-value > 0.05 (типичное пороговое значение), то делаем вывод о том, что взаимосвязь между временем пользования аккаунтом и количеством обращений в колл-центр статистически незначима.

Какие ещё инсайты можно найти в имеющихся данных? Творческое задание.

Рассмотрим зависимости между общей длительностью звонков, суммой и длительностью.

In [ ]:
df['total calls'] = df['total day calls'] + df['total eve calls'] + df['total night calls'] + df['total intl calls']

In [ ]:
df['total charge'] = df['total day charge'] + df['total eve charge'] + df['total night charge'] + df['total intl charge']

In [ ]:
test = ['total calls', 'total minutes', 'total charge']
sns.pairplot(df[test]);

In [ ]:
df[test].corr(method='spearman')

In [ ]:
sns.heatmap(df[test].corr(method='spearman'));

Как и ожидалась зависимость между 'total minutes' и 'total charge' достаточно высока.

Проверим зависят ли принаки 'account length' и 'churn'.

In [ ]:
df.groupby('churn')['account length'].mean().plot(kind='bar') 
plt.ylabel('account length') 
plt.show();

In [ ]:
df.groupby('churn')['account length'].hist()
plt.xlabel('account length') 
plt.ylabel('number of subscribers')
plt.show();

In [ ]:
# Применяем бисериальный коэффициент корреляции.

pb1 = pointbiserialr(df['account length'], df['churn'])
print('Point biserialr correlation:', pb1[0], 'p-value:', pb1[1]) 

Теория не подтвердилась p-value не принимает малое значение, следовательно связь между 'account length' и 'churn' не является статистически значимой.